# Coursera Final Capstone Project: Battle of the Neighborhoods

This capstone project will be graded by your peers. This capstone project is worth 70% of your total grade. The project will be completed over the course of 2 weeks. Week 1 submissions will be worth 30% whereas week 2 submissions will be worth 40% of your total grade.

**For this week, you will required to submit the following:**

    A description of the problem and a discussion of the background. (15 marks)
    A description of the data and how it will be used to solve the problem. (15 marks)

**For the second week, the final deliverables of the project will be:**

    A link to your Notebook on your Github repository, showing your code. (15 marks)
    A full report consisting of all of the following components (15 marks):

    Introduction where you discuss the business problem and who would be interested in this project.
    Data where you describe the data that will be used to solve the problem and the source of the data.
    Methodology section which represents the main component of the report where you discuss and describe any exploratory data analysis that you did, any inferential statistical testing that you performed, if any, and what machine learnings were used and why.
    Results section where you discuss the results.
    Discussion section where you discuss any observations you noted and any recommendations you can make based on the results.
    Conclusion section where you conclude the report.

    Your choice of a presentation or blogpost. (10 marks)

### Background

Hypothetical scenario: data scientist, John Doe, currently lives in California, but he finds a remote data science jobs that would allow him to relocate, and California is too expensive.  He hears good things about the neighboring state, Arizona.

But John Doe really likes the feel of his current neighbhorhood / current town, in terms of venues nearby: parks, stores, restaurants, etc.

### Goal

Create a notebook that does the following:
1. Allow an input city for comparision (where John Doe currently lives)
2. Load a csv file that compiles suburbs in the Phoenix, AZ metropolitan area (created manually from wiki page: https://en.wikipedia.org/wiki/Phoenix_metropolitan_area)
3. Use FourSquare data to gather venue statistics about both the current input city and ALL Phoenix community cities
4. Use clustering models to determine which city near Phoenix is most similar to John's current city, as a nice location to move to.

### Data

Most of the data will come from Foursquare API.  Only other data is a csv file created manually, which just lists out all the communities (with population > 10,000) in the Phoenix, AZ metropolitan area.  List 